In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.utils.data as data
from torchsummary import summary
from tqdm import tqdm, trange
from torch.utils.data import random_split, SubsetRandomSampler, DataLoader
from sklearn.model_selection import KFold
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import models
import glob
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import albumentations as A
from albumentations.pytorch import ToTensorV2
import math
import numpy as np

C:\Users\user\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_num_threads(2)
# Micro French-English Dictinary 
translate = {"cane": "dog", 
             "cavallo": "horse",
             "elefante": "elephant",
             "farfalla": "butterfly",
             "gallina": "chicken", 
             "gatto": "cat", 
             "mucca": "cow", 
             "pecora": "sheep",
             "ragno": "spider",
             "scoiattolo": "squirrel"
            }

translate_class2num= {
             "cane": 0, 
             "cavallo": 1,
             "elefante": 2,
             "farfalla": 3,
             "gallina": 4, 
             "gatto": 5, 
             "mucca": 6, 
             "pecora": 7,
             "ragno": 8,
             "scoiattolo": 9
            }

translate_num2slasseng = {
             0: "dog", 
             1: "horse",
             2: "elephant",
             3: "butterfly",
             4: "chicken", 
             5: "cat", 
             6: "cow", 
             7: "sheep",
             8: "spider",
             9: "squirrel"
            }

In [3]:
train_path = r"C:\Users\user\Documents\DL_Practice\dataset\animal10\train"
test_path =  r"C:\Users\user\Documents\DL_Practice\dataset\animal10\test"
batch_size= 64
num_epochs= 50
learning_rate = 0.001

In [4]:
classes = translate.keys()

for _class in classes:
    print(translate[_class], len(os.listdir(train_path +'/'+ _class)))

dog 3890
horse 2098
elephant 1156
butterfly 1689
chicken 2478
cat 1334
cow 1492
sheep 1456
spider 3856
squirrel 1489


In [5]:
img_path_list = []
img_classes_list = []
skiped_list = []
for class_ in classes: 
    for img in os.listdir(train_path+'/'+ class_):
        if (img[-4:] != "jpeg"):
            skiped_list.append(img)
            continue
        img_path = train_path+'/' + class_ + "/" + img
        img_path_list.append(img_path)
        img_classes_list.append(translate_class2num[class_])
        
print(len(img_path_list), len(img_classes_list))
print("Skipped:", len(skiped_list))
#and (img[-3:] != "jpg")
pd.value_counts(img_classes_list)

19301 19301
Skipped: 1637


0    3890
8    3532
4    2478
1    2098
6    1492
9    1489
3    1227
5    1226
7    1080
2     789
dtype: int64

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(img_path_list, img_classes_list, test_size=0.1, random_state=42)
print(len(x_train), len(x_valid))

17370 1931


In [7]:
img_path_list = []
img_classes_list = []
skiped_list = []
for class_ in classes: 
    for img in os.listdir(test_path +'/'+ class_):
        if (img[-4:] != "jpeg"):
            skiped_list.append(img)
            continue
        img_path = test_path +'/' + class_ + "/" + img
        img_path_list.append(img_path)
        img_classes_list.append(translate_class2num[class_])
        
print(len(img_path_list), len(img_classes_list))
print("Skipped:", len(skiped_list))
#and (img[-3:] != "jpg")
pd.value_counts(img_classes_list)
x_test,y_test = img_path_list, img_classes_list

4908 4908
Skipped: 333


In [8]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

preprocess_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

class AnimalsDataset(Dataset):
    
    def __init__(self, imgs_list, class_list, transforms = None):
        
        super().__init__()
        self.imgs_list = imgs_list
        self.class_list = class_list
        self.transforms = transforms
        
        
    def __getitem__(self, index):
    
        image_path = self.imgs_list[index]
        input_image = Image.open(image_path)

        if self.transforms:
            image = self.transforms(input_image)   

        label = torch.from_numpy(np.array(self.class_list[index])).to(torch.int64)
        
        return image, label    
        
    def __len__(self):
        return len(self.imgs_list)


In [9]:
class CustomDatasetDataLoader():
    def name(self):
        return 'CustomDatasetDataLoader'
    def __init__(self, imgs_list, class_list, transforms = None):
        # super().__init__()
        self.dataset = AnimalsDataset(imgs_list, class_list, transforms)
        self.dataloader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True)

    def load_data(self):
        return self.dataloader

    def __len__(self):
        return len(self.dataset)

In [10]:
def CreateDataLoader(imgs_list, class_list, transforms = None):
    data_loader = CustomDatasetDataLoader(imgs_list, class_list, transforms)
    print(data_loader.name())
    dataloader = data_loader.load_data()
    return dataloader

In [11]:
train_data_loader = CreateDataLoader(x_train, y_train, preprocess)
valid_data_loader = CreateDataLoader(x_valid, y_valid, preprocess_valid)
test_data_loader = CreateDataLoader(x_test, y_test, preprocess_valid)
print(len(train_data_loader),len(valid_data_loader),len(test_data_loader))


CustomDatasetDataLoader
CustomDatasetDataLoader
CustomDatasetDataLoader
272 31 77


In [12]:
# Iterations number
iter_num =  math.ceil(len(train_data_loader) / batch_size)
iter_valid_num =  math.ceil(len(valid_data_loader) / batch_size)
print(iter_num, iter_valid_num)

5 1


In [13]:
# res_18_model = models.resnet18(pretrained=True)
# num_fc_ftr = res_18_model.fc.in_features #获取到fc层的输入
# res_18_model.fc = nn.Linear(num_fc_ftr, 10)
# model=res_18_model

In [14]:
class SE_VGG16(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        # define an empty for Conv_ReLU_MaxPool
        net = []

        # block 1
        net.append(nn.Conv2d(in_channels=3, out_channels=64, padding=1, kernel_size=3, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=64, out_channels=64, padding=1, kernel_size=3, stride=1))
        net.append(nn.ReLU())
        net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # block 2
        net.append(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1))
        net.append(nn.ReLU())
        net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # block 3
        net.append(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # block 4
        net.append(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # block 5
        net.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1))
        net.append(nn.ReLU())
        net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # add net into class property
        self.features = nn.Sequential(*net)

        # define an empty container for Linear operations
        classifier = []
        classifier.append(nn.Linear(in_features=512*7*7, out_features=4096))
        classifier.append(nn.ReLU())
        classifier.append(nn.Dropout(p=0.5))
        classifier.append(nn.Linear(in_features=4096, out_features=4096))
        classifier.append(nn.ReLU())
        classifier.append(nn.Dropout(p=0.5))
        classifier.append(nn.Linear(in_features=4096, out_features=self.num_classes))

        # add classifier into class property
        self.classifier = nn.Sequential(*classifier)


    def forward(self, x):
        features = self.features(x)
        features = features.view(x.size(0), -1)
        classify_result = self.classifier(features)
        return classify_result

In [15]:
# VGG = models.vgg16(pretrained=True)
# mod = list(VGG.classifier.children())
# mod.pop()
# mod.append(torch.nn.Linear(4096, 10))
# new_classifier = torch.nn.Sequential(*mod)
# print(list(list(new_classifier.children())[1].parameters()))
# VGG.classifier = new_classifier
# for param_tensor in VGG.state_dict():
#     print(param_tensor, "\t", VGG.state_dict()[param_tensor].size())

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.save(VGG.state_dict(), 'VGG16.pth')
model = SE_VGG16(num_classes=10)
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
model.load_state_dict(torch.load('VGG16.pth'))
model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
summary(model, (3, 224, 224))

features.0.weight 	 torch.Size([64, 3, 3, 3])
features.0.bias 	 torch.Size([64])
features.2.weight 	 torch.Size([64, 64, 3, 3])
features.2.bias 	 torch.Size([64])
features.5.weight 	 torch.Size([128, 64, 3, 3])
features.5.bias 	 torch.Size([128])
features.7.weight 	 torch.Size([128, 128, 3, 3])
features.7.bias 	 torch.Size([128])
features.10.weight 	 torch.Size([256, 128, 3, 3])
features.10.bias 	 torch.Size([256])
features.12.weight 	 torch.Size([256, 256, 3, 3])
features.12.bias 	 torch.Size([256])
features.14.weight 	 torch.Size([256, 256, 3, 3])
features.14.bias 	 torch.Size([256])
features.17.weight 	 torch.Size([512, 256, 3, 3])
features.17.bias 	 torch.Size([512])
features.19.weight 	 torch.Size([512, 512, 3, 3])
features.19.bias 	 torch.Size([512])
features.21.weight 	 torch.Size([512, 512, 3, 3])
features.21.bias 	 torch.Size([512])
features.24.weight 	 torch.Size([512, 512, 3, 3])
features.24.bias 	 torch.Size([512])
features.26.weight 	 torch.Size([512, 512, 3, 3])
features.

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 7, 7]           --
|    └─Conv2d: 2-1                       [-1, 64, 224, 224]        1,792
|    └─ReLU: 2-2                         [-1, 64, 224, 224]        --
|    └─Conv2d: 2-3                       [-1, 64, 224, 224]        36,928
|    └─ReLU: 2-4                         [-1, 64, 224, 224]        --
|    └─MaxPool2d: 2-5                    [-1, 64, 112, 112]        --
|    └─Conv2d: 2-6                       [-1, 128, 112, 112]       73,856
|    └─ReLU: 2-7                         [-1, 128, 112, 112]       --
|    └─Conv2d: 2-8                       [-1, 128, 112, 112]       147,584
|    └─ReLU: 2-9                         [-1, 128, 112, 112]       --
|    └─MaxPool2d: 2-10                   [-1, 128, 56, 56]         --
|    └─Conv2d: 2-11                      [-1, 256, 56, 56]         295,168
|    └─ReLU: 2-12                        [-1, 256, 56, 56]      

In [18]:
## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [19]:
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    epoch_acc=0.0
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # Sets the model in training mode.
    epoch_loss, epoch_correct = 0, 0

    for batch_i, (images, labels) in tqdm(enumerate(dataloader)):
        images, labels = images.to(device), labels.to(device) # move data to GPU

        # Compute prediction loss
        pred = model(images)
        loss = criterion(pred, labels)

        # Optimization by gradients
        optimizer.zero_grad() # set prevision gradient to 0
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        #epoch_loss += loss.item()
        epoch_loss += loss.detach().item()
        # epoch_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        epoch_correct +=(torch.max(pred, 1)[1].view(labels.size()).data == labels.data).sum()
        epoch_acc += get_accuracy(pred, labels, batch_size)
    
    
    return epoch_loss/num_batches,epoch_acc/len(dataloader)

In [21]:
def test(dataloader, model, loss_fn):
    epoch_acc=0.0
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # Sets the model in test mode.
    epoch_loss, epoch_correct = 0, 0

    # No training for test data
    with torch.no_grad():
        for batch_i, (images, labels) in tqdm(enumerate(dataloader)):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)
            loss = criterion(pred, labels)
            
            #epoch_loss += loss.item()
            epoch_loss += loss.item()
            # epoch_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            epoch_correct+=(torch.max(pred, 1)[1].view(labels.size()).data == labels.data).sum()
            epoch_acc += get_accuracy(pred, labels, batch_size)
    return epoch_loss/num_batches, epoch_acc/len(dataloader)

In [22]:
# define draw
def plotCurve(x_vals, y_vals, 
                x_label, y_label, 
                x2_vals=None, y2_vals=None, 
                legend=None,
                figsize=(3.5, 2.5)):
    # set figsize
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.plot(x_vals, y_vals)
    # plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        # plt.semilogy(x2_vals, y2_vals, linestyle=':')
        plt.plot(x_vals, y2_vals)
    if legend:
        plt.legend(legend)
    plt.show()

In [23]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(train_data_loader, model, criterion, optimizer)
    val_loss, val_acc = test(valid_data_loader, model, criterion)
    print("Epoch:{}/{}  Training Loss:{:.3f} || Training Acc {:.3f} % ||  valid Loss:{:.3f} ||  valid Acc {:.3f} %".format(epoch + 1,
                                                                                                         num_epochs,
                                                                                                         train_loss,
                                                                                                         train_acc,                            
                                                                                                         val_loss,
                                                                                                         val_acc))

272it [02:20,  1.94it/s]
31it [00:07,  4.11it/s]


Epoch:1/50  Training Loss:1.960 || Training Acc 31.893 % ||  valid Loss:1.654 ||  valid Acc 43.498 %


272it [02:19,  1.95it/s]
31it [00:07,  4.07it/s]


Epoch:2/50  Training Loss:1.636 || Training Acc 43.825 % ||  valid Loss:1.407 ||  valid Acc 50.252 %


272it [02:19,  1.95it/s]
31it [00:07,  4.09it/s]


...

Epoch:48/50  Training Loss:0.783 || Training Acc 74.242 % ||  valid Loss:0.718 ||  valid Acc 75.756 %


272it [02:17,  1.98it/s]
31it [00:07,  4.13it/s]


Epoch:49/50  Training Loss:0.764 || Training Acc 74.627 % ||  valid Loss:0.705 ||  valid Acc 75.706 %


272it [02:17,  1.98it/s]
31it [00:07,  4.13it/s]

Epoch:50/50  Training Loss:0.786 || Training Acc 73.983 % ||  valid Loss:0.718 ||  valid Acc 74.546 %


In [24]:
test_loss, test_acc = test(test_data_loader, model, criterion)
print(f'Fold, test acc: {test_acc:.3f}')

77it [00:19,  3.88it/s]

Fold, test acc: 76.096
